In [4]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification
from datasets import load_dataset


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [6]:

model_path = r"D:\Zoro_project2\zero-short-text-classification\models\bert_sst2_model"
model = BertForSequenceClassification.from_pretrained(model_path).to(device)
tokenizer = BertTokenizer.from_pretrained(model_path)


In [7]:
# Load IMDb dataset
imdb = load_dataset("imdb")
test_data = imdb["test"].shuffle(seed=42).select(range(50))   # sample 50 for quick test



d:\Zoro_project2\zero-short-text-classification\py_3\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Jyoti\.cache\huggingface\hub\datasets--imdb. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Generating unsupervised split: 100%|██████████| 50000/50000 [00:00<00:00, 94818.84 examples/s] 


In [8]:
# Zero-Shot Prediction
def predict_zero_shot(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=256).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
        probs = torch.softmax(outputs.logits, dim=-1)
        pred = torch.argmax(probs, dim=-1).item()
    return "positive" if pred == 1 else "negative"

zero_shot_results = []
for example in test_data:
    zero_shot_results.append({
        "text": example["text"][:200] + "...",
        "true_label": "positive" if example["label"]==1 else "negative",
        "pred_label": predict_zero_shot(example["text"])
    })
df_zero = pd.DataFrame(zero_shot_results)
df_zero.to_csv(r"D:\Zoro_project2\zero-short-text-classification\outputs\sst2\zero_shot_predictions_imdb.csv", index=False)


In [ ]:
#  Few-Shot Predictions

few_shot_examples = [
    ("This movie was fantastic, I loved it!", "positive"),
    ("The film was terrible and boring.", "negative"),
    ("An amazing story with great acting.", "positive"),
]

def predict_few_shot(text, examples=few_shot_examples):
    # Note: BERT is discriminative, not generative → examples don’t change input encoding
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=256).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
        probs = torch.softmax(outputs.logits, dim=-1)
        pred = torch.argmax(probs, dim=-1).item()
    return "positive" if pred == 1 else "negative"

few_shot_results = []
for example in test_data:
    few_shot_results.append({
        "text": example["text"][:200] + "...",
        "true_label": "positive" if example["label"]==1 else "negative",
        "pred_label": predict_few_shot(example["text"]) 
    })  

df_few = pd.DataFrame(few_shot_results)
df_few.to_csv(r"D:\Zoro_project2\zero-short-text-classification\outputs\sst2\few_shot_predictions_imdb.csv", index=False)
print(" Few-shot predictions saved as few_shot_predictions_imdb.csv")


 Few-shot predictions saved as few_shot_predictions_imdb.csv


In [10]:
#  Accuracy Comparison

zero_acc = (df_zero["true_label"] == df_zero["pred_label"]).mean()
few_acc = (df_few["true_label"] == df_few["pred_label"]).mean()

print(f"🎯 Zero-Shot Accuracy: {zero_acc:.2f}")
print(f"🎯 Few-Shot Accuracy: {few_acc:.2f}")

🎯 Zero-Shot Accuracy: 0.96
🎯 Few-Shot Accuracy: 0.96
